In [1]:
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model, Input
from keras.layers import Dense, Dropout
from keras.layers import Embedding, Activation, Flatten
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D, BatchNormalization
from keras.utils import to_categorical
from keras import optimizers
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import time


Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
def folder_to_df(letter):
    full_path ="data/bonn_uni_datasets/"+ letter + "/*.*"
    files = files = glob.glob(full_path)
    df_list = []
    for file in files:
        df_list.append(pd.read_csv(file, header = None))
    big_df = pd.concat(df_list, ignore_index=True, axis= 1)
    return big_df.T

def norm(X):
    X = X - np.mean(X)
    X = X / np.std(X)
    return X

def window(a, w = 512, o = 64, copy = False):
    #default for training, for testing data we split each signal in four of 1024 and apply
    #a window size of 512 with a stride (o) of 256
    sh = (a.size - w + 1, w)
    st = a.strides * 2
    view = np.lib.stride_tricks.as_strided(a, strides = st, shape = sh)[0::o]
    if copy:
        return view.copy()
    else:
        return view

def enrich_train(df):
    labels = df.iloc[:,-1]
    data = df.iloc[:, :-1]
    res = list()
    for i in range(len(data)):
        res += [window(data.iloc[i].values)]
    return res


In [3]:
def load_data_as_df():
    A = norm(folder_to_df('A'))
    B = norm(folder_to_df('B'))
    C = norm(folder_to_df('C'))
    D = norm(folder_to_df('D'))
    E = norm(folder_to_df('E'))
    
    normal = A.append(B).reset_index(drop = True)
    interictal = C.append(D).reset_index(drop = True)
    ictal = E

    return normal, interictal, ictal


In [4]:
normal, interictal, ictal = load_data_as_df()

In [5]:
normal_train, normal_vote = train_test_split(normal, test_size = 0.1)
interictal_train, interictal_vote = train_test_split(interictal, test_size = 0.1)
ictal_train, ictal_vote = train_test_split(ictal, test_size = 0.1)

In [6]:
def format_enrich_train(normal, interictal, ictal):
    
    normal_train_enr = np.asarray(enrich_train(normal)).reshape(-1, np.asarray(enrich_train(normal)).shape[-1])
    interictal_train_enr = np.asarray(enrich_train(interictal)).reshape(-1, np.asarray(enrich_train(interictal)).shape[-1])
    ictal_train_enr = np.asarray(enrich_train(ictal)).reshape(-1, np.asarray(enrich_train(ictal)).shape[-1])

    normal_train_enr_df = pd.DataFrame(normal_train_enr)
    interictal_train_enr_df = pd.DataFrame(interictal_train_enr)
    ictal_train_enr_df = pd.DataFrame(ictal_train_enr)
    
    normal_train_enr_df['labels'] = 0 # normal
    interictal_train_enr_df['labels'] = 1 #interictal
    ictal_train_enr_df['labels'] = 2 #ictal

    data_labels = pd.concat([normal_train_enr_df, interictal_train_enr_df, ictal_train_enr_df], ignore_index = True)
    

    
    data = data_labels.drop('labels', axis = 1).values
    labels = data_labels.labels.values
    
    #labels = np.expand_dims(labels, axis=1)
    
    return data, labels

In [7]:
def create_model():
    model = Sequential()
    #Conv - 1
    model.add(Conv1D(24, 5,strides =  3, input_shape=(512,1)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    #Conv - 2
    model.add(Conv1D(16, 3,strides =  2))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    #Conv - 3
    model.add(Conv1D(8, 3,strides =  2))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    #FC -1
    model.add(Flatten())
    model.add(Dense(20))
    model.add(Activation('relu'))
    #Dropout
    model.add(Dropout(0.5))
    #FC -2
    model.add(Dense(3,activation = 'softmax'))
    #softmax
    #model.add(Activation('softmax'))

    adam = optimizers.Adam(lr=0.00002, beta_1=0.9, beta_2=0.999, epsilon=0.00000001, decay=0.0, amsgrad=False)

    model.compile(loss='categorical_crossentropy',
                  optimizer=adam,
                  metrics=['accuracy'])
    return model

In [14]:
def train_evaluate_model(model, xtrain, ytrain, xval, yval):
    history = model.fit(xtrain, ytrain, batch_size=32, epochs=200)
    score = model.evaluate(xval, yval, batch_size=32)
    return score, history

In [9]:
def reshape_x(arr):
    nrows = arr.shape[0]
    ncols = arr.shape[1]
    return arr.reshape(nrows, ncols, 1)

def reshape_y(arr):
    nrows = arr.shape[0]
    ncols = 1
    return arr.reshape(nrows, ncols, 1)
    

In [15]:
n_folds = 10
X, y = format_enrich_train(normal, interictal, ictal)
skf = StratifiedKFold(n_splits=10, shuffle=True)



for i, (train, test) in enumerate(skf.split(X,y)):
    print("Running Fold", i+1, "/", n_folds)
    start_time = time.time()
    X = reshape_x(X)
    xtrain, xval = X[train], X[test]
    ytrain, yval = y[train], y[test]
    ytrain = to_categorical(ytrain, num_classes=3, dtype='float32')
    yval = to_categorical(yval, num_classes=3, dtype='float32')


    model = None # Clearing the NN.
    model = create_model()
    score, history = train_evaluate_model(model, xtrain, ytrain, xval, yval)
    print("Ran ", i+1, "/", n_folds, "Fold in %s seconds ---" % (time.time() - start_time))
    print(score)
    print('\n')
    print(history)

Running Fold 1 / 10
Epoch 1/200
25650/25650 [==============================] - 11s 417us/step - loss: 1.2490 - acc: 0.3503
Epoch 2/200
25650/25650 [==============================] - 11s 422us/step - loss: 1.0827 - acc: 0.4514
Epoch 3/200
25650/25650 [==============================] - 11s 419us/step - loss: 1.0040 - acc: 0.5423
Epoch 4/200
25650/25650 [==============================] - 11s 422us/step - loss: 0.9315 - acc: 0.6054
Epoch 5/200
25650/25650 [==============================] - 11s 424us/step - loss: 0.8766 - acc: 0.6390
Epoch 6/200
25650/25650 [==============================] - 11s 423us/step - loss: 0.8400 - acc: 0.6532
Epoch 7/200
25650/25650 [==============================] - 11s 421us/step - loss: 0.8128 - acc: 0.6670
Epoch 8/200
25650/25650 [==============================] - 11s 422us/step - loss: 0.7975 - acc: 0.6731
Epoch 9/200
25650/25650 [==============================] - 11s 424us/step - loss: 0.7795 - acc: 0.6824
Epoch 10/200
25650/25650 [===========================

25650/25650 [==============================] - 11s 427us/step - loss: 0.1940 - acc: 0.9368
Epoch 80/200
25650/25650 [==============================] - 11s 424us/step - loss: 0.1949 - acc: 0.9373
Epoch 81/200
25650/25650 [==============================] - 11s 443us/step - loss: 0.1855 - acc: 0.9397
Epoch 82/200
25650/25650 [==============================] - 11s 425us/step - loss: 0.1886 - acc: 0.9377
Epoch 83/200
25650/25650 [==============================] - 11s 430us/step - loss: 0.1853 - acc: 0.9398
Epoch 84/200
25650/25650 [==============================] - 11s 447us/step - loss: 0.1794 - acc: 0.9419
Epoch 85/200
25650/25650 [==============================] - 11s 440us/step - loss: 0.1831 - acc: 0.9407
Epoch 86/200
25650/25650 [==============================] - 11s 432us/step - loss: 0.1782 - acc: 0.9428
Epoch 87/200
25650/25650 [==============================] - 11s 438us/step - loss: 0.1775 - acc: 0.9428
Epoch 88/200
25650/25650 [==============================] - 11s 420us/step - 

25650/25650 [==============================] - 11s 409us/step - loss: 0.1223 - acc: 0.9601
Epoch 158/200
25650/25650 [==============================] - 11s 412us/step - loss: 0.1283 - acc: 0.9599
Epoch 159/200
25650/25650 [==============================] - 11s 416us/step - loss: 0.1246 - acc: 0.9600
Epoch 160/200
25650/25650 [==============================] - 11s 414us/step - loss: 0.1262 - acc: 0.9587
Epoch 161/200
25650/25650 [==============================] - 11s 415us/step - loss: 0.1222 - acc: 0.9606
Epoch 162/200
25650/25650 [==============================] - 10s 400us/step - loss: 0.1240 - acc: 0.9592
Epoch 163/200
25650/25650 [==============================] - 10s 405us/step - loss: 0.1241 - acc: 0.9599
Epoch 164/200
25650/25650 [==============================] - 10s 405us/step - loss: 0.1250 - acc: 0.9589
Epoch 165/200
25650/25650 [==============================] - 10s 403us/step - loss: 0.1262 - acc: 0.9592
Epoch 166/200
25650/25650 [==============================] - 10s 407u

KeyboardInterrupt: 